In [1]:
import pandas as pd
import sqlite3
import os

# --- 1. 設定 ---
DB_PATH = "data/processed/s2orc_filtered.db"
EVAL_PAPERS_FILE = "data/datapapers/sampled/evaluation_data_papers_50.csv"

print("--- Checking Evaluation Ground Truth ---")

# --- 2. 評価用データ論文（50件）のリストをロード ---
try:
    df_eval_papers = pd.read_csv(EVAL_PAPERS_FILE)
    eval_data_paper_dois = tuple(df_eval_papers['cited_datapaper_doi'].unique())
    print(f"Loaded {len(eval_data_paper_dois)} data papers from {EVAL_PAPERS_FILE}")
except Exception as e:
    print(f"Error loading CSV: {e}")
    raise

# --- 3. DBを調査 ---
results = []
try:
    with sqlite3.connect(DB_PATH) as conn:
        for data_paper_doi in eval_data_paper_dois:
            # 正解DOI (Human=1) の数をカウント
            query_gt = "SELECT COUNT(citing_doi) FROM positive_candidates WHERE cited_datapaper_doi = ? AND human_annotation_status = 1"
            count = conn.execute(query_gt, (data_paper_doi,)).fetchone()[0]
            results.append({
                'data_paper_doi': data_paper_doi,
                'human_used_count': count
            })

    # --- 4. 結果の表示 ---
    df_results = pd.DataFrame(results)
    
    # 正解が0件のものを表示
    df_missing = df_results[df_results['human_used_count'] == 0]
    
    print("\n" + "="*50)
    if not df_missing.empty:
        print(f"Found {len(df_missing)} data papers with ZERO 'Human: Used' (human_status=1) entries:")
        print(df_missing)
    else:
        print("All 50 evaluation data papers have at least 1 'Human: Used' entry.")
    print("="*50)

except Exception as e:
    print(f"An error occurred: {e}")

--- Checking Evaluation Ground Truth ---
Loaded 50 data papers from data/datapapers/sampled/evaluation_data_papers_50.csv

Found 2 data papers with ZERO 'Human: Used' (human_status=1) entries:
               data_paper_doi  human_used_count
15  10.1016/J.DIB.2016.05.025                 0
33  10.1016/J.DIB.2018.11.111                 0


In [3]:
import pandas as pd
import sqlite3
import os
import numpy as np
# ▼▼▼ tabulateが不要な 'display' をインポート ▼▼▼
from IPython.display import display

# --- 1. 設定 ---
DB_PATH = "data/processed/s2orc_filtered.db"
EVAL_PAPERS_FILE = "data/datapapers/sampled/evaluation_data_papers_50.csv"
TRAINING_PAPERS_FILE = "data/datapapers/sampled/training_data_papers_50.csv"

N_REPLACEMENTS_NEEDED = 2 # 2件不足している

print("--- Finding Replacement Evaluation Papers (Top-K Cited) ---")

try:
    with sqlite3.connect(DB_PATH) as conn:
        # 1. 元の母集団（LLM=Usedが2件以上）の全リストを取得
        query = """
            SELECT
                cited_datapaper_doi,
                COUNT(citing_doi) AS llm_used_count
            FROM
                positive_candidates
            WHERE
                llm_annotation_status = 1
            GROUP BY
                cited_datapaper_doi
            HAVING
                COUNT(citing_doi) >= 2;
        """
        df_all_eligible = pd.read_sql_query(query, conn)
        all_eligible_dois = set(df_all_eligible['cited_datapaper_doi'])
        print(f"Total eligible pool size: {len(all_eligible_dois)}")

    # 2. 既に使用済みのDOIをロード
    df_eval = pd.read_csv(EVAL_PAPERS_FILE)
    eval_dois = set(df_eval['cited_datapaper_doi'])
    
    df_train = pd.read_csv(TRAINING_PAPERS_FILE)
    train_dois = set(df_train['cited_datapaper_doi'])
    
    used_dois = eval_dois.union(train_dois)
    print(f"Already used DOIs (Train + Eval): {len(used_dois)}")

    # 3. 未使用のDOIプールを作成
    available_dois = list(all_eligible_dois - used_dois)
    df_available = df_all_eligible[df_all_eligible['cited_datapaper_doi'].isin(available_dois)]
    
    if len(df_available) < N_REPLACEMENTS_NEEDED:
        print(f"❌ Error: Not enough available papers in the pool to find {N_REPLACEMENTS_NEEDED} replacements.")
    else:
        # 4. 被引用数（llm_used_count）が多い順にソートし、上位2件を取得
        replacements_df = df_available.sort_values(by='llm_used_count', ascending=False).head(N_REPLACEMENTS_NEEDED)

        print("\n" + "="*50)
        print(f"✅ Found {N_REPLACEMENTS_NEEDED} replacement data papers (Top {N_REPLACEMENTS_NEEDED} by LLM 'Used' count):")
        
        # ▼▼▼ 修正点: .to_markdown() を display() に変更 ▼▼▼
        display(replacements_df)
        print("="*50)
        
        # 5. 元のCSVファイルに追記
        print(f"Appending these {N_REPLACEMENTS_NEEDED} papers to {EVAL_PAPERS_FILE}...")
        replacements_df.to_csv(EVAL_PAPERS_FILE, mode='a', header=False, index=False)
        
        df_new_eval = pd.read_csv(EVAL_PAPERS_FILE)
        print(f"Successfully updated. New evaluation set size: {len(df_new_eval)} papers.")
        
        print("\n--- ❗ 次のステップ ❗ ---")
        print("1. `evaluation_data_papers_50.csv` が 52件 に更新されました。")
        print("2. `annotator_app` を `ANNOTATION_MODE = \"evaluation\"` に設定して起動し、")
        print("   今追加された 2件 の論文に紐づく候補を**目視でアノテーション**してください。")
        print("3. アノテーションが完了し、新しい2件に `human_status=1` の正解が作成されたら、")
        print("   評価スクリプト (`22b_...ipynb`) を再実行してください。")
        print("   (今度は 50件中50件 が処理されるはずです)")

except Exception as e:
    print(f"💥 An error occurred: {e}")

--- Finding Replacement Evaluation Papers (Top-K Cited) ---
Total eligible pool size: 721
Already used DOIs (Train + Eval): 200

✅ Found 2 replacement data papers (Top 2 by LLM 'Used' count):


,cited_datapaper_doi,llm_used_count
0,10.1002/ECY.2663,3
364,10.1038/S41597-019-0217-0,3


Appending these 2 papers to data/datapapers/sampled/evaluation_data_papers_50.csv...
Successfully updated. New evaluation set size: 52 papers.

--- ❗ 次のステップ ❗ ---
1. `evaluation_data_papers_50.csv` が 52件 に更新されました。
2. `annotator_app` を `ANNOTATION_MODE = "evaluation"` に設定して起動し、
   今追加された 2件 の論文に紐づく候補を**目視でアノテーション**してください。
3. アノテーションが完了し、新しい2件に `human_status=1` の正解が作成されたら、
   評価スクリプト (`22b_...ipynb`) を再実行してください。
   (今度は 50件中50件 が処理されるはずです)


In [ ]:
import pandas as pd
import sqlite3
import os
from IPython.display import display

# --- 1. 設定 ---
DB_PATH = "data/processed/s2orc_filtered.db"

# ▼▼▼ どのモードのデータを一括承認するか選択 ▼▼▼
# 'evaluation'     : 評価用データ（約50グループ）の未処理分
# 'training'       : 訓練用データ（150グループ）の未処理分
# 'training_advanced': 訓練用トップ20の未処理分
UPDATE_MODE = "evaluation" 
# ▲▲▲ --------------------------------- ▲▲▲

# アノテーション対象のDOIリストを読み込む
EVAL_PAPERS_FILE = "data/datapapers/sampled/evaluation_data_papers_50.csv"
TRAINING_PAPERS_FILE = "data/datapapers/sampled/training_data_papers_50.csv"

print(f"Target mode set to: {UPDATE_MODE}")

In [ ]:
# --- 2. 対象DOIリストの読み込み ---
TARGET_DOI_LIST = ()
try:
    if UPDATE_MODE == "evaluation":
        df_papers = pd.read_csv(EVAL_PAPERS_FILE)
        TARGET_DOI_LIST = tuple(df_papers['cited_datapaper_doi'].str.upper().tolist())
    elif UPDATE_MODE == "training":
        df_papers = pd.read_csv(TRAINING_PAPERS_FILE)
        TARGET_DOI_LIST = tuple(df_papers['cited_datapaper_doi'].str.upper().tolist())
    elif UPDATE_MODE == "training_advanced":
        df_papers = pd.read_csv(TRAINING_PAPERS_FILE)
        df_top_20 = df_papers.nlargest(20, 'used_paper_count')
        TARGET_DOI_LIST = tuple(df_top_20['cited_datapaper_doi'].str.upper().tolist())
    else:
        raise ValueError(f"Unknown UPDATE_MODE: {UPDATE_MODE}")
        
    print(f"Loaded {len(TARGET_DOI_LIST)} data paper DOIs for mode '{UPDATE_MODE}'.")

except Exception as e:
    print(f"An error occurred loading DOI lists: {e}")

In [ ]:
# --- 3. データベースの一括更新 ---
if TARGET_DOI_LIST:
    print("Connecting to database...")
    try:
        with sqlite3.connect(DB_PATH) as conn:
            cursor = conn.cursor()
            
            # 1. 更新対象の件数を事前にカウント
            placeholders = ','.join('?' for _ in TARGET_DOI_LIST)
            count_query = f"""
                SELECT COUNT(*)
                FROM positive_candidates
                WHERE cited_datapaper_doi IN ({placeholders})
                  AND llm_annotation_status = 1
                  AND human_annotation_status = 0
            """
            # (TARGET_DOI_LISTを2回渡す必要があるため、リストを結合)
            params = TARGET_DOI_LIST
            count_before = cursor.execute(count_query, params).fetchone()[0]
            
            if count_before == 0:
                print("No unprocessed items found. Database is already up to date.")
            else:
                print(f"Found {count_before:,} items to approve (set human_status=1)...")
                
                # 2. 一括更新（UPDATE）の実行
                update_query = f"""
                    UPDATE positive_candidates
                    SET human_annotation_status = 1
                    WHERE cited_datapaper_doi IN ({placeholders})
                      AND llm_annotation_status = 1
                      AND human_annotation_status = 0
                """
                
                cursor.execute(update_query, params)
                updated_rows = cursor.rowcount # 実際に更新された行数
                conn.commit()
                
                print("\n" + "="*50)
                print(f"✅ Successfully updated {updated_rows:,} rows.")
                print(f"Mode '{UPDATE_MODE}' is now fully annotated.")
                print("="*50)

    except Exception as e:
        print(f"💥 An error occurred during database update: {e}")
else:
    print("No target DOIs loaded. Script stopped.")